# Python and ArcPy for LANDFIRE in Fire Needs Assessments 

## Setup and Packages

### Overview

This section helps you set up your Python environment for the Fire Needs Assessment workflow. You’ll start by installing and loading the necessary packages, and then define your global environment settings that will be used throughout the workflow.

---

This code was created using ArcGIS Pro version 3.2.2 and will require the Spatial Analyst extension. If you are working in a different version of the software or do not have a Spatial Analyst license, some of these tools may not work as expected.

---

Before you begin, you will need to:

- Set up an ArcGIS Online project called FNA
- Create three directories, “Rasters.gdb”, and “Outputs”
- Download the file “https://tnc.box.com/s/nnuwixwc4ot21iovuj0zvq1f750hxh6e” (clicking will take you to a box file. Click Download to initiate a download that will likely land in your “C:” directory). This zip file is large (5.13GB) and will likely take a long time to download. Please plan your space and download time accordingly. That will download the Inputs.gdb, which contains:

    - A feature class of New York State
    - A CONUS BpS raster
    - A CONUS EVT raster
    
- Extract the “fna_python_ag.zip” files into your FNA project.
- Create and save a new python-script with a name such as “fna_python_code”.

Once this set up is complete, you should be able to copy/paste the code below into the python script you created above or run the code in the below section if you are following along with our tutorial.

In [12]:
#Required only when there are SubModel(s)
import sys 
sys.path.append(r"C:\Users\shagen\AppData\Local\Temp\ArcGISProTemp32472")
# -*- coding: utf-8 -*-

import arcpy
from arcpy.sa import *
from arcpy.sa import *
from sys import argv

import os
os.chdir("D:\\LANDFIRE\\GIS\\GIS_Projects\\Current\\FNA\\Python")

# =======================================================
# Define Global Environment Settings
# =======================================================
# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("spatial")

# Set the snap raster environment
arcpy.env.snapRaster = ".\\Inputs.gdb\\BpS"

# Define Cell Size
arcpy.env.cellSize = "30"

# Define Output Coordinate System
arcpy.env.outputCoordinateSystem = "PROJCS[\"NAD_1983_Contiguous_USA_Albers\",GEOGCS[\"GCS_North_American_1983\",DATUM[\"D_North_American_1983\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Albers\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-96.0],PARAMETER[\"Standard_Parallel_1\",29.5],PARAMETER[\"Standard_Parallel_2\",45.5],PARAMETER[\"Latitude_Of_Origin\",23.0],UNIT[\"Meter\",1.0]]"

# Also set the default scratch and output workspaces globally
arcpy.env.scratchWorkspace = ".\\Rasters.gdb"
arcpy.env.workspace = ".\\Rasters.gdb"
# =======================================================

## Step 1: Clip/Extract the LANDFIRE rasters to your analysis extent

### Overview
This step clips the LANDFIRE Biophysical Settings (BpS) and Existing Vegetation Type (EVT) rasters to your analysis boundary using the `ExtractByMask` tool. This ensures that subsequent processing focuses only on your area of interest.

### Prerequisites
- ArcGIS Pro 3.2.2 or later with the **Spatial Analyst extension** enabled.
- Project directories set up:
  - `Inputs.gdb` containing the CONUS BpS and EVT rasters and your boundary feature class.
  - `Rasters.gdb` for intermediate outputs.
  - `Outputs` folder for final tables.
- Global environment settings configured:
  - `arcpy.env.snapRaster` set to the CONUS BpS raster.
  - `arcpy.env.cellSize` set to 30 meters.
  - `arcpy.env.outputCoordinateSystem` set to NAD 1983 Albers (CONUS).

### This code will:
- Clip the BpS raster to the boundary feature class and save it as `NY_BpS` in `Rasters.gdb`.
- Clip the EVT raster to the same boundary and save it as `NY_EVT` in `Rasters.gdb`.

**Note**
- If you do not have the Spatial Analyst extension, you can use the Raster Clip tool instead, but subsequent steps require Spatial Analyst.
- Ensure your boundary feature class and rasters share the same projection before running this step.


In [2]:
# Step 1: Clip/Extract LANDFIRE rasters to analysis extent
# Set model inputs. Replace with your own rasters and your own boundary if you are not using the demo.
Boundary=".\\Inputs.gdb\\NY_boundary"
BpS_Raster=".\\Inputs.gdb\\BpS"
Masked_BpS=".\\Rasters.gdb\\NY_BpS"
EVT_Raster=".\\Inputs.gdb\\EVT"
Masked_EVT=".\\Rasters.gdb\\NY_EVT"

#Boundary=".\\Inputs.gdb\\NY_boundary", BpS_Raster=".\\Inputs.gdb\\BpS", Masked_BpS=".\\Rasters.gdb\\NY_BpS", EVT_Raster=".\\Inputs.gdb\\EVT", Masked_EVT=".\\Rasters.gdb\\NY_EVT"

# For inline variable substitution, parameters passed as a String are evaluated using locals(), globals() and isinstance(). To override, substitute values directly.
def Extract_to_Boundary():  # Extract LANDFIRE rasters to boundary.

    # Process: Extract BpS to boundary (Extract by Mask) (sa)
    Extract_BpS_to_boundary = Masked_BpS
    try:
        print(f"Attempting Extract by Mask for BPS raster: {BpS_Raster}")
        arcpy.sa.ExtractByMask(BpS_Raster, Boundary).save(Masked_BpS)
        print(f"BPS data successfully masked and saved to {Masked_BpS}")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in BpS Process.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in BpS process: {e}")


    # Process: Extract EVT to boundary (Extract by Mask) (sa)
    Extract_EVT_to_boundary = Masked_EVT
    try:
        print(f"Attempting Extract by Mask for EVT raster: {EVT_Raster}")
        arcpy.sa.ExtractByMask(EVT_Raster, Boundary).save(Masked_EVT)
        print(f"EVT data successfully masked and saved to {Masked_EVT}")

    except arcpy.ExecuteError:
        print("Failed: ArcPy Execution Error in EVT Process.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in BpS process: {e}")

    print("Extract_to_Boundary execution finished.")

if __name__ == '__main__':
    # When the script is run directly, this calls the main function Extract_to_Boundary
    Extract_to_Boundary()


Attempting Extract by Mask for BPS raster: .\Inputs.gdb\BpS
BPS data successfully masked and saved to .\Rasters.gdb\NY_BpS
Attempting Extract by Mask for EVT raster: .\Inputs.gdb\EVT
EVT data successfully masked and saved to .\Rasters.gdb\NY_EVT
Extract_to_Boundary execution finished.



## Step 2: Examining Existing Vegetation Types

Assessing current fire needs involves understanding the current vegetation types and adjusting historical fire return intervals to reflect current management practices. This section focuses on processing LANDFIRE Existing Vegetation Type (EVT) data using Python and ArcPy.

---

### A Note on Fire Return Intervals
We are still using historical fire return intervals, even though we are analyzing current systems. Current fire return intervals may differ from those for the underlying Biophysical Settings. Later, we will share examples of how others have updated these intervals. For now, the goal is to understand how historical intervals relate to current vegetation so we can begin refining fire needs estimates within the analysis area.

---


### Overview
This step filters out non-vegetated areas from the EVT raster and uses the resulting burnable zones to mask the Biophysical Settings (BPS) raster. This ensures that fire return interval calculations are limited to ecologically relevant areas.

### This code will:
- Define a SQL where clause to exclude non-burnable classes (e.g., Agricultural, Developed, Open Water).
- Apply the `ExtractByAttributes` function to the masked EVT raster using the specified where clause.
- Save the resulting raster of burnable EVT types to the project geodatabase.


---

Ensure the Spatial Analyst extension is available and the input raster includes the `EVT_PHYS` field for filtering.

---



In [3]:
# Step 2: Mask EVT by attributes
# Set model inputs. Replace with your own rasters if you are not using the demo.
Where_clause="EVT_PHYS <> 'Agricultural' And EVT_PHYS <> 'Developed' And EVT_PHYS <> 'Developed-High Intensity' And EVT_PHYS <> 'Developed-Low Intensity' And EVT_PHYS <> 'Developed-Medium Intensity' And EVT_PHYS <> 'Developed-Roads' And EVT_PHYS <> 'NA' And EVT_PHYS <> 'Open Water' And EVT_PHYS <> 'Quarries-Strip Mines-Gravel Pits-Well and Wind Pads'" # Update these to include or exclude EVT values as desired
Burnable_EVTs=".//Rasters.gdb//NY_EVT_Burnable"

#Masked_EVT, Where_clause, Burnable_EVTs
#For inline variable substitution, parameters passed as a String are evaluated using locals(), globals() and isinstance(). To override, substitute values directly.
def Extract_EVT():

    # Process: Extract EVT by Attributes (Extract by Attributes) (sa)
    Extract_EVT_by_Attributes = Burnable_EVTs
    try:
        print(f"Attempting Extract by Attributes for EVT raster: {Masked_EVT}")
        arcpy.sa.ExtractByAttributes(Masked_EVT, Where_clause).save(Burnable_EVTs)
        print(f"EVT raster successfully masked and saved to {Burnable_EVTs}")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in EVT Mask.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in EVT mask: {e}")


if __name__ == '__main__':
    # When the script is run directly, this calls the main function Extract_EVT_by_Attributes
    Extract_EVT()


Attempting Extract by Attributes for EVT raster: .\Rasters.gdb\NY_EVT
EVT raster successfully masked and saved to .//Rasters.gdb//NY_EVT_Burnable



## Step 3: Mask the BPS Raster Using Burnable EVT

### Overview
Assessing fire needs requires refining the analysis to focus only on ecologically relevant areas. After creating the burnable EVT raster in the previous step, we now use it to mask the Biophysical Settings (BPS) raster. This ensures that fire return interval calculations apply only to zones where vegetation can burn.

### This code will:
- Load the masked BPS raster (`Masked_BPS`) and the burnable EVT raster (`Burnable_EVTs`) from the project geodatabase.
- Apply a mask to the BPS raster using the burnable EVT raster (`ExtractByMask`) to produce `Burnable_BPSs`.
- Add fields `Acres` and `Acres_Year` to the output raster’s table **if they are missing**.
- Calculate `Acres` from `COUNT` using `COUNT × 900 × 0.0002471` *(assumes 30 m pixels)*.
- If the interval field (`FRI_ALLFIR`) exists, calculate `Acres_Year` as `Acres / FRI_ALLFIR`.
- Save the masked BPS raster to the project geodatabase.


In [13]:
# Step 3: Mask BPS by EVT
# Set model inputs. Replace with your own rasters if you are not using the demo.
Masked_BPS = r".//Rasters.gdb//NY_BpS"      # input raster to be masked
Burnable_EVTs=r".//Rasters.gdb//NY_EVT_Burnable"   # mask raster
Burnable_BPSs = r".//Rasters.gdb//NY_BPS_Burnable"  # output raster after masking

#For inline variable substitution, parameters passed as a String are evaluated using locals(), globals() and isinstance(). To override, substitute values directly.
def Extract_BPS():
    
    # Process: Extract BPS by mask (Extract by Mask) (sa)
    Extract_BPS_by_mask = Burnable_BPSs
    try:
        print(f"Attempting Extract by Mask for BPS raster: {Masked_BPS}")
        arcpy.sa.ExtractByMask(Masked_BPS, Burnable_EVTs).save(Burnable_BPSs)
        print(f"BPS raster successfully masked and saved to {Burnable_BPSs}")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in BPS Mask.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in BPS mask: {e}")

    # Add fields (Acres, Acres_Year) if missing
    def field_exists(table, name):
        return any(f.name.lower() == name.lower() for f in arcpy.ListFields(table))

    for fld, alias in [("Acres","Acres"), ("Acres_Year","Acres per Year")]:
        if field_exists(Burnable_BPSs, fld):
            print(f"Field '{fld}' already exists—skipping.")
        else:
            print(f"Adding field '{fld}'...")
            arcpy.management.AddField(Burnable_BPSs, fld, "DOUBLE", field_alias=alias)
            print(f"Added field '{fld}'.")

# Calculate Acres from COUNT (30 m pixels)
    try:
        arcpy.management.CalculateField(
            in_table=Burnable_BPSs,
            field="Acres",
            expression="!COUNT! * 900 * 0.0002471",
            expression_type="PYTHON3"
        )
        print("Calculated 'Acres' using COUNT * 900 * 0.0002471.")
    except arcpy.ExecuteError:
        print("ArcPy error calculating 'Acres'.")
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(f"General error calculating 'Acres': {e}")

    # Calculate Acres_Year if your interval field exists
    FRI_FIELD = "FRI_ALLFIR"  # your interval field
    try:
        if field_exists(Burnable_BPSs, FRI_FIELD):
            print(f"Calculating 'Acres_Year' = !Acres! / !{FRI_FIELD}!")
            arcpy.management.CalculateField(
                in_table=Burnable_BPSs,
                field="Acres_Year",
                expression=f"!Acres! / !{FRI_FIELD}!",
                expression_type="PYTHON3"
            )
            print(f"Calculated 'Acres_Year' using '{FRI_FIELD}'.")
        else:
            print(f"Skipped 'Acres_Year': '{FRI_FIELD}' field not found.")
    except arcpy.ExecuteError:
        print("ArcPy error calculating 'Acres_Year'.")
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(f"General error calculating 'Acres_Year': {e}")


if __name__ == '__main__':
    # When the script is run directly, this calls the main function Extract_BPS_by_EVT
    Extract_BPS()

Attempting Extract by Mask for BPS raster: .//Rasters.gdb//NY_BpS
BPS raster successfully masked and saved to .//Rasters.gdb//NY_BPS_Burnable
Adding field 'Acres'...
Added field 'Acres'.
Adding field 'Acres_Year'...
Added field 'Acres_Year'.
Calculated 'Acres' using COUNT * 900 * 0.0002471.
Calculating 'Acres_Year' = !Acres! / !FRI_ALLFIR!
Calculated 'Acres_Year' using 'FRI_ALLFIR'.



## Step 4: Combine Burnable BPS and Burnable EVT

### Overview
Understanding how Biophysical Settings (BPS), Existing Vegetation Type (EVT), and historical fire return intervals interact on the landscape requires combining the burnable BPS and EVT rasters. This step combines the BPS and EVT rasters into a single raster that links each unique BPS–EVT pair to a unique ID. This combined raster will be used in later steps to join attributes and calculate fire return metrics for ecologically relevant areas.

### This code will:
- Load the burnable BPS raster (`Burnable_BPSs`) and burnable EVT raster (`Burnable_EVTs`) from the project geodatabase.
- Use the `Combine` function to merge both rasters into a single raster (`Combined_BPS_EVT`).
- Save the combined raster to the project geodatabase.


In [17]:
# Step 4: Combine Burnable BPS and Burnable EVT
# Set model inputs. Replace with your own rasters if you are not using the demo.
Burnable_BPSs = r".//Rasters.gdb//NY_BPS_Burnable"  # input BPS raster
Burnable_EVTs = r".//Rasters.gdb//NY_EVT_Burnable"   # input EVT raster
Combined_BPS_EVT = r".//Rasters.gdb//NY_BPS_EVT_Burnable"  # output raster after combine

#For inline variable substitution, parameters passed as a String are evaluated using locals(), globals() and isinstance(). To override, substitute values directly.
def Combine_BPS_EVT():
    # Process: Combine (Combine) (sa)
    Combine_BPS_EVT = Combined_BPS_EVT
    try:
        print(f"Attempting combine for : {Burnable_BPSs} and {Burnable_EVTs}")
        arcpy.sa.Combine([Burnable_BPSs, Burnable_EVTs]).save(Combined_BPS_EVT)
        print(f"BPS raster successfully masked and saved to {Combined_BPS_EVT}")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in combine.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in combine: {e}")

if __name__ == '__main__':
    # When the script is run directly, this calls the main function Extract_BPS_by_EVT
    Combine_BPS_EVT()

Attempting combine for : .//Rasters.gdb//NY_BPS_Burnable and .//Rasters.gdb//NY_EVT_Burnable
BPS raster successfully masked and saved to .//Rasters.gdb//NY_BPS_EVT_Burnable



## Step 5: Join Attributes and Export Combined Table

### Overview
The combined raster currently contains only basic fields (e.g., Object ID, Value, Count, and the value fields from BPS and EVT). To make this data useful for analysis, we need to join additional attributes from the original BPS and EVT rasters and then export the table for further processing. This table will serve as the foundation for calculating fire return metrics and summarizing results by vegetation type.

### This code will:
- Join BPS attributes to the combined raster using `JoinField`:
  - Fields include: `BPS_NAME`, `BPS_MODEL`, `GROUPVEG`, fire return interval fields (`FRI_ALLFIR`, `FRI_REPLAC`, `FRI_MIXED`, `FRI_SURFAC`), fire severity percentages, and acreage fields (`Acres`, `Acres_Year`).
- Join EVT attributes to the combined raster using `JoinField`:
  - Fields include: `EVT_NAME`, `EVT_PHYS`, `EVT_GP_N`.
- Export the updated raster attribute table to a CSV file for further analysis.

**Note**
- Ensure the combined raster and join tables exist in the specified geodatabase.
- Update field lists if your analysis requires additional attributes.
- The exported table will be saved to the `Outputs` folder as a `.csv` file.


In [22]:
# Step 5: Join fields and export table
# Set model inputs. Replace with your own files if you are not using the demo.
Combined_BPS_EVT_Raster = r".//Rasters.gdb//NY_BPS_EVT_Burnable" # input raster to which table will be joined
BPS_Field = "NY_BPS_Burnable"
BPS_Join_Table = r".//Rasters.gdb//NY_BPS_Burnable"
BPS_Join_Field = "value"
BPS_Transfer_Fields = ["BPS_NAME", "BPS_MODEL", "GROUPVEG", "FRI_ALLFIR", "FRI_REPLAC", "FRI_MIXED", "FRI_SURFAC", "PRC_REPLAC", "PRC_MIXED", "PRC_SURFAC", "FRG_NEW", "Acres", "Acres_Year"]
EVT_Field = "NY_EVT_Burnable"
EVT_Join_Table = r".//Rasters.gdb//NY_EVT_Burnable"
EVT_Join_Field = "value"
EVT_Transfer_Fields = ["EVT_NAME", "EVT_PHYS", "EVT_GP_N"]
Combined_BPS_EVT_Table = r".//Outputs//NY_BPS_EVT_Burnable.csv"


#For inline variable substitution, parameters passed as a String are evaluated using locals(), globals() and isinstance(). To override, substitute values directly.
def Join_BPS_Variables():  # Model
    # Process: Join BPS Variables (Join Field) (management)
    try:
        print(f"Attempting join field for : {Combined_BPS_EVT_Raster} and {BPS_Join_Table}")
        arcpy.management.JoinField(Combined_BPS_EVT_Raster, BPS_Field, BPS_Join_Table, BPS_Join_Field, BPS_Transfer_Fields, fm_option="NOT_USE_FM", index_join_fields="NO_INDEXES")
        print(f"BPS fields successfully joined")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in combine.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in join fields: {e}")                    

    # Process: Join EVT variables (Join Field) (management)
    try:
        print(f"Attempting join field for : {Combined_BPS_EVT_Raster} and {EVT_Join_Table}")
        arcpy.management.JoinField(Combined_BPS_EVT_Raster, EVT_Field, EVT_Join_Table, EVT_Join_Field, EVT_Transfer_Fields, fm_option="NOT_USE_FM", index_join_fields="NO_INDEXES")
        print(f"BPS fields successfully joined")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in combine.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in join fields: {e}")
    

    # Process: Export Table (Export Table) (conversion)
    try:
        print(f"Attempting to export: {Combined_BPS_EVT_Table}")
        arcpy.conversion.ExportTable(Combined_BPS_EVT_Raster, Combined_BPS_EVT_Table, use_field_alias_as_name="NOT_USE_ALIAS")
        print(f"table successfully exported to {Combined_BPS_EVT_Table}")

    except arcpy.ExecuteError:
        print(f"Failed: ArcPy Execution Error in combine.")
        print(arcpy.GetMessages(2)) # prints error messages
    except Exception as e:
        # catches other potential Python errors (e.g. indentation, syntax)
        print(f"Failed: A general error occurred in join fields: {e}")

if __name__ == '__main__':
    # When the script is run directly, this calls the main function Join_BPS_Variables
    Join_BPS_Variables()

Attempting join field for : .//Rasters.gdb//NY_BPS_EVT_Burnable and .//Rasters.gdb//NY_BPS_Burnable
BPS fields successfully joined
Attempting join field for : .//Rasters.gdb//NY_BPS_EVT_Burnable and .//Rasters.gdb//NY_EVT_Burnable
BPS fields successfully joined
Attempting to export: .//Outputs//NY_BPS_EVT_Burnable.csv
table successfully exported to .//Outputs//NY_BPS_EVT_Burnable.csv



## Next Steps: Working with the Exported Table

The combined BPS–EVT table has been exported as a CSV file. From this point forward, the workflow for analyzing and summarizing the data is the same as in the ArcGIS Pro and R tutorials.

To continue:

### If you prefer Excel  
Follow the instructions starting with [Working with the CSV File](https://landfire-tnc.github.io/LANDFIRE_FNA/ArcGIS/tabular-csv.html) in the Fire Needs Assessments in ArcGIS Pro section of the [LANDFIRE in Fire Needs Assessments website](https://landfire-tnc.github.io/LANDFIRE_FNA/).

### If you prefer R  
Follow the instructions starting with [Working with the Tabular Data](https://landfire-tnc.github.io/LANDFIRE_FNA/r-workflow/r-tabular.html#calculating-evt-acres-in-r) in the Fire Needs Assessments in R section of the [LANDFIRE in Fire Needs Assessments website](https://landfire-tnc.github.io/LANDFIRE_FNA/).

These resources will guide you through:
- Reviewing and cleaning the table,
- Calculating fire return metrics,
- Summarizing results by vegetation type.

**Note:**  
The Python workflow ends here. All subsequent steps for data manipulation and analysis should follow the Excel or R workflows linked above.
